In [1]:
import warnings # default warn => SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=UserWarning)
import pandas as pd; pd.options.mode.chained_assignment = None
import numpy as np
from datetime import datetime, date
import time
import scipy.fftpack
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt
import arrow
from pprint import pprint as pp
#from tqdm import tqdm

import dmyplant2
from dmyplant2 import (
    cred, MyPlant, Engine,
    FSMOperator, startstopFSM, FSM_splot, FSM_splotBC, FSM_VLine, FSM_add_Notations, FSM_add_Alarms, FSM_add_Warnings,
    bokeh_show, dbokeh_chart, add_dbokeh_vlines, get_cycle_data2, disp_result, disp_alarms, disp_warnings,
    cvset, cplotdef, equal_adjust, count_columns, load_data, get_cycle_data, get_cycle_data2, figures)

import ipywidgets as widgets
from ipywidgets import AppLayout, Button, Layout, VBox, HBox, Label, HTML, interact, interact_manual, interactive, IntSlider, Output
from IPython.display import HTML, display
display(HTML("<style>.container {width:94% !important;}</style>"))

from bokeh.io import push_notebook, show, output_notebook
from bokeh.models import ColumnDataSource, Label, Text, Span, HoverTool #, Range1d#, LabelSet
from bokeh.plotting import figure
output_notebook(hide_banner=True)

# login to myplant()
cred()
mp = MyPlant(0)
Engine._list_cached_validations()
#mp._fetch_installed_base(); # refresh local installed fleet database

,1486152,1330175
Asset ID,159397,114517
Validation Engine,Forsa Hartmoor M02,CREYKE BECK M03
n,999,999
oph@start,54.0,0
serialNumber,1486152,1330175
starts@start,57.0,0
val start,2022-01-10 00:00:00,2017-12-05 00:00:00
source,from_MyPlant,from_MyPlant


In [2]:
def sfun(x):
    return all([ ("CREYKE BECK" in str(x['IB Site Name'])),  (x['OperationalCondition'] != 'Decommissioned') ])
    #return all([ ("Yukon Energy" in str(x['IB Site Name'])),  (x['OperationalCondition'] != 'Decommissioned') ])
    #return all([ ("Forsa Hartmoor" in str(x['IB Site Name'])),  (x['OperationalCondition'] != 'Decommissioned') ])
    #return all([ ("EWB" in str(x['IB Site Name'])),  (x['OperationalCondition'] != 'Decommissioned') ])
    #return all([ ("Pforzheim" in str(x['IB Site Name'])),  (x['OperationalCondition'] != 'Decommissioned') ])
    #return all([ ("BMW Landshut 4.10" in str(x['IB Site Name'])),  (x['OperationalCondition'] != 'Decommissioned') ])
fleet = mp.search_installed_fleet(sfun).drop('index', axis=1)
fleet = fleet.sort_values(by = "Engine ID",ascending=True).reset_index(drop='index')
fleet.T

,0,1,2,3,4,5,6,7,8,9,10
Count_OpHour,7584.0,7592.0,7627.0,7609.0,7601.0,7506.0,7593.0,7596.0,7601.0,7599.0,7597.0
OperationalCondition,Available,Available,Available,Available,Available,Available,Available,Available,Available,Available,Available
shutdown_counter,3840.0,3767.0,3904.0,3851.0,3724.0,3818.0,3713.0,3757.0,3738.0,3779.0,3736.0
startup_counter,3840.0,3767.0,3904.0,3850.0,3725.0,3818.0,3713.0,3756.0,3738.0,3778.0,3735.0
id,114432,114433,114517,114516,114434,114435,114533,114951,115074,115249,115250
Engine Version,H312,H312,H312,H312,H312,H312,H312,H312,H312,H312,H312
Engine Type,624,624,624,624,624,624,624,624,624,624,624
Engine Series,6,6,6,6,6,6,6,6,6,6,6
Country,GB,GB,GB,GB,GB,GB,GB,GB,GB,GB,GB
Commissioning Date,2018-02-05,2017-12-05,2017-12-05,2017-12-05,2017-12-05,2017-12-05,2017-12-05,2017-12-05,2017-12-05,2017-12-05,2017-12-05


In [3]:
motor = fleet.iloc[2]
modes = ['undefined','OFF','MAN','AUTO']
success = ['success','failed','undefined']
e=Engine.from_fleet(mp,motor)
#pp_from='2022-03-28 06:00' # 1 Start
#pp_to='2022-03-28 08:14'
pp_from='2023-03-27'
pp_to='2023-03-28'
motor['IB Site Name'] + ' ' + motor['Engine ID'], modes, success

('CREYKE BECK M03',
 ['undefined', 'OFF', 'MAN', 'AUTO'],
 ['success', 'failed', 'undefined'])

In [5]:
fsm = FSMOperator(e, p_from=pp_from, p_to=pp_to)
fsm.run0(enforce=True, silent=False, debug=False)
fsm.run1(silent=False, successtime=300, debug=False) # run Finite State Machine
fsm.run2(silent=False)
fsm.run4(silent=False)

FSM4: 100%|██████████████████████████████████| 3/3 [00:00<00:00, 17.66 starts/s]


In [7]:
pp_from='2023-03-28'
pp_to='2023-03-29'
fsm2 = FSMOperator(e, p_from=pp_from, p_to=pp_to)
fsm2.run0(enforce=True, silent=False, debug=False)
fsm2.run1(silent=False, successtime=300, debug=False) # run Finite State Machine
fsm2.run2(silent=False)
fsm2.run4(silent=False)

FSM4: 100%|██████████████████████████████████| 3/3 [00:00<00:00, 16.35 starts/s]


In [10]:
print(f"FSM  {fsm.results['starts_counter']} {fsm.results['first_message']} {fsm.results['last_message']}")
print(f"FSM2 {fsm2.results['starts_counter']} {fsm2.results['first_message']} {fsm2.results['last_message']}")


FSM  3 2023-03-27 00:23:24.297999872 2023-03-27 23:23:29.032999936
FSM2 3 2023-03-28 00:23:29.184999936 2023-03-28 23:23:28.510000128


In [11]:
pp_from='2023-03-27'
pp_to='2023-03-29'
fsm3 = FSMOperator(e, p_from=pp_from, p_to=pp_to)
fsm3.run0(enforce=True, silent=False, debug=False)
fsm3.run1(silent=False, successtime=300, debug=False) # run Finite State Machine
fsm3.run2(silent=False)
fsm3.run4(silent=False)
print(f"FSM3 {fsm3.results['starts_counter']} {fsm3.results['first_message']} {fsm3.results['last_message']}")


FSM4: 100%|██████████████████████████████████| 6/6 [00:00<00:00, 14.94 starts/s]
FSM3 6 2023-03-27 00:23:24.297999872 2023-03-28 23:23:28.510000128


In [ ]:
rdf = fsm.starts
rda = rdf[:].reset_index(drop='index')
rda = rda[(rda['mode'].isin(modes) & rda['success'].isin(success))].reset_index(drop='index')
rdb = rda
von_dt=pd.to_datetime(fsm.first_message); von=int(von_dt.timestamp())
bis_dt=pd.to_datetime(fsm.last_message); bis=int(bis_dt.timestamp())
print(f"von: {von_dt.strftime('%d.%m.%Y %H:%M:%S')} --- bis: {bis_dt.strftime('%d.%m.%Y %H:%M:%S')}")
print(f"Starts: {rdf.shape[0]}, Successful: {rdf[rdf['success'] == 'success'].shape[0]}, Failed: {rdf[rdf['success'] == 'failed'].shape[0]}, Undefined: {rdf[rdf['success'] == 'undefined'].shape[0]} => ")
print(f"Starting reliability raw: {rdf[rdf['success'] == 'success'].shape[0]/(rdf.shape[0])*100.0:3.1f}% ")
print(f"Starting reliability: {rdf[rdf['success'] == 'success'].shape[0]/(rdf.shape[0]-rdf[rdf['success'] == 'undefined'].shape[0])*100.0:3.1f}% ")
pd.DataFrame.from_dict(e.dash, orient='index').T

In [ ]:
#rda = rda[rda['A'] > 0]
rda[startstopFSM.run2filter_content].round(2).fillna('')

In [ ]:
rda[startstopFSM.run4filter_content].round(2).fillna('')

In [ ]:
#rde = rda[(rda.starttime > fsm._e['Commissioning Date']) & (rda['success']) & (rda.targetload > 2800.0)].copy()
#rde = rda[(rda.starttime > fsm._e['Commissioning Date']) & (rda.A > 0)].copy()
#rde = rda[(rda.starttime > fsm._e['Commissioning Date']) & (rda['success'])].copy()
rde = rda.fillna(0)
rde['datetime'] = pd.to_datetime(rde['starttime'])
sdict ={'success':1, 'failed':0, 'undefined':0.5}
rde['isuccess'] = rde.apply(lambda x: sdict[x['success']], axis=1)
vec = ['oilfilling','degasing','speedup','idle','synchronize','loadramp','targetload','ramprate','cumstarttime','targetoperation','rampdown','coolrun','runout','isuccess']
display(rde[vec].describe().round(2))
dfigsize = (18,8)
dset = [
    {'col':['cumstarttime'],'_ylim':(-600,800), 'color':'darkblue'},
    {'col':['runout'],'_ylim':(0,100) },
    {'col':['targetload'],'_ylim':(-4000,26000) },
    {'col':['ramprate'],'_ylim':(-5,7)},
    {'col':['loadramp'],'_ylim':(-150,900), 'color':'red'},
    {'col':['speedup'],'_ylim':(-100,200), 'color':'orange'},
    {'col':['synchronize'],'_ylim':(-20,400)},
    {'col':['oilfilling','degasing'],'_ylim':(-1000,800)},
    {'col':['W','A','isuccess'],'_ylim':(-1,200), 'color':['rgba(255,165,0,0.3)','rgba(255,0,0,0.3)','rgba(0,128,0,0.2)'] },
    {'col':['no'],'_ylim':(0,1000), 'color':['rgba(0,0,0,0.1)'] },
    ]
dset = equal_adjust(dset, rde, do_not_adjust=[-1], debug=False)
ftitle = f"{fsm._e}"
fig = dbokeh_chart(rde, dset, style='both', figsize=dfigsize ,title=ftitle);
bokeh_show(fig)

In [ ]:
out = widgets.Output()
pfigsize=(16,8)

def myfigures(e):
    return {
    'actors' : [
        {'col':['Power_SetPower','Power_PowerAct'], 'ylim':(0,5000), 'color':['lightblue','red'], 'unit':'kW'},
        {'col':['Various_Values_SpeedAct'],'ylim': [0, 2500], 'color':'blue', 'unit':'rpm'},
        {'col':['Ignition_ITPAvg'],'ylim': [-10, 30], 'color':'rgba(255,0,255,0.4)', 'unit':'°KW'},
        {'col':['TecJet_Lambda1'],'ylim': [0, 3], 'color':'rgba(255,165,0,0.4)', 'unit':'-'},
        {'col':['Various_Values_PosThrottle','Various_Values_PosTurboBypass'],'ylim': [-10, 110], 'color':['rgba(105,105,105,0.6)','rgba(165,42,42,0.4)'], 'unit':'%'},
        ],
    'tecjet' : [
        {'col':['Power_SetPower','Power_PowerAct'], 'ylim':(0,5000), 'color':['lightblue','red'], 'unit':'kW'},
        {'col':['Various_Values_SpeedAct'],'ylim': [0, 2500], 'color':'blue', 'unit':'rpm'},
        {'col':['TecJet_Lambda1'],'ylim': [0, 3], 'color':'rgba(255,165,0,0.4)', 'unit':'-'},
        {'col':['TecJet_GasPress1'],'_ylim': [0, 3], 'color':'rgba(255,0,0,0.4)', 'unit':'mbar'},
        {'col':['TecJet_GasTemp1'],'_ylim': [0, 3], 'color':'rgba(255,0,255,0.4)', 'unit':'°C'},
        {'col':['TecJet_GasDiffPress'],'_ylim': [0, 3], 'color':'rgba(0,255,0,0.4)', 'unit':'mbar'},
        ],
    'lubrication' : [
        {'col':['Power_SetPower','Power_PowerAct'], 'ylim':(0,5000), 'color':['lightblue','red'], 'unit':'kW'},
        {'col':['Various_Values_SpeedAct'],'ylim': [0, 2500], 'color':'blue', 'unit':'rpm'},
        {'col':['Hyd_PressCrankCase'],'ylim': [-100, 100], 'color':'orange', 'unit':'mbar'},
        {'col':['Hyd_PressOilDif'],'ylim': [0, 3], 'color':'black', 'unit': 'bar'},
        {'col':['Hyd_PressOil'],'ylim': [0, 10], 'color':'brown', 'unit': 'bar'},
        {'col':['Hyd_TempOil','Hyd_TempCoolWat','Hyd_TempWatRetCoolOut'],'ylim': [0, 110], 'color':['#2171b5','orangered','hotpink'], 'unit':'°C'},
        ],
    'exhaust' : [
        {'col':['Power_SetPower','Power_PowerAct'], 'ylim':(0,5000), 'color':['lightblue','red'], 'unit':'kW'},
        {'col':['Various_Values_SpeedAct'],'ylim': [0, 2500], 'color':'blue', 'unit':'rpm'},
        {'col':['TecJet_Lambda1'],'ylim': [0, 3], 'color':'rgba(255,165,0,0.4)', 'unit':'-'},
        {'col':e.dataItemsCyl('Exhaust_TempCyl*'),'ylim': [400, 700], 'unit':'°C'},
        {'col':e.dataItemsCyl('Knock_Valve_Noise_Cyl*'),'ylim': [0, 12000], 'unit':'mV'},
        ],
    'ignition' : [
        {'col':['Power_SetPower','Power_PowerAct'], 'ylim':(0,5000), 'color':['lightblue','red'], 'unit':'kW'},
        {'col':['Various_Values_SpeedAct'],'ylim': [0, 2500], 'color':'blue', 'unit':'rpm'},
        {'col':['TecJet_Lambda1'],'ylim': [0, 3], 'color':'rgba(255,165,0,0.4)', 'unit':'-'},
        {'col':e.dataItemsCyl('Monic_VoltCyl*'),'ylim': [0, 100], 'unit':'kV'},
        {'col':e.dataItemsCyl('Ignition_ITPCyl*'),'ylim': [0, 40], 'unit':'°KW'},
        {'col':e.dataItemsCyl('Knock_KLS98_IntKnock_Cyl*'),'ylim': [-30, 60], 'unit':'%'},
        ],    
    }

def update_fig(x=0):
    global vv; 
    rdbs = rdb[rdb.no == x]
    if not rdbs.empty:
        startversuch = rdbs.iloc[0]
        #startversuch = rdb.iloc[x]
        vv = startversuch.no; 
        #ftitle = f"{fsm._e} ----- Start {startversuch['no']} {startversuch['mode']} | {'SUCCESS' if startversuch['success'] else 'FAILED'} | {startversuch['starttime'].round('S')} CumStart: {startversuch['cumstarttime']:0.1f}"
        #display(HTML(ftitle));
        #disp_result(startversuch)

        # PLotter
        lfigures = myfigures(fsm._e)
        plotdef, vset = cplotdef(mp, lfigures)
        #dmaxlength = 1800
        dmaxlength = None
        dminlength = None
        #startversuch = rdb.iloc[vv]
        ftitle = f"{fsm._e} ----- Start {startversuch['no']} {startversuch['mode']} | {'SUCCESS' if startversuch['success'] else 'FAILED'} | {startversuch['starttime'].round('S')}"
        data = get_cycle_data2(fsm, startversuch, max_length=dmaxlength, min_length=dminlength, cycletime=1, silent=True, p_data=vset)
        data['power_diff'] = pd.Series(np.gradient(data['Power_PowerAct']))
        fig_handles = []
        for doplot in plotdef:
            dset = lfigures[doplot]
            ltitle = f"{ftitle} | {doplot}"
            if count_columns(dset) > 12: # no legend, if too many lines.
                fig = FSM_splot(fsm, startversuch, data, dset, title=ltitle, legend=False, figsize=pfigsize)
            else:
                fig = FSM_splot(fsm, startversuch, data, dset, title=ltitle, figsize=pfigsize)

            fig = FSM_add_Notations(fig, fsm, startversuch)
            disp_alarms(startversuch)
            disp_warnings(startversuch)
            fig = FSM_add_Alarms(fig, fsm, startversuch)
            fig = FSM_add_Warnings(fig, fsm, startversuch)
            fig_handles.append(bokeh_show(fig, notebook_handle=True))
        for h in fig_handles:
            push_notebook(handle=h)
    else:
        print(f"Start No {x} is not in the filtered results. please consider changing filters, if you want to access it.")

In [ ]:
vv = 0
@out.capture(clear_output=True)
def cb(but):
    update_fig(_a.value)

def html_summary(*args):
    sv = rdf.iloc[_a.value]
    global vv; vv = _a.value;
    s = '''
    <style>
        table, td, th {
            border: 1px solid grey;
            border-collapse: collapse;
            padding: 0px 5px;
            text-align: right;
        }
    </style>
    '''
    summary = pd.DataFrame(sv[startstopFSM.run2filter_content]).T
    r = f"{s}{summary.to_html(escape=False, index=False)}"
    _e.value = r
    #return r 
    
_a = widgets.IntText(description='No: ',layout=widgets.Layout(width='10%'))
_a.observe(html_summary, 'value')
_b = widgets.IntSlider(0, 0, rdf.shape[0]-1 , 1, layout=widgets.Layout(width='70%'))
mylink = widgets.jslink((_a, 'value'), (_b, 'value'))
_d = widgets.Button(description='Show Plots',disabled=False, button_style='primary')
_d.on_click(cb)
_e = widgets.HTML()
html_summary()

display( widgets.VBox([widgets.HBox([_b, _a, _d]),_e, out]));

In [ ]:
update_fig(vv)

In [ ]:
%matplotlib widget
vv = 0
@out.capture(clear_output=True)
def cb(but):
    update_fig(_a.value)

def html_summary(*args):
    sv = rdf.iloc[_a.value]
    global vv; vv = _a.value;
    s = '''
    <style>
        table, td, th {
            border: 1px solid grey;
            border-collapse: collapse;
            padding: 0px 5px;
            text-align: right;
        }
    </style>
    '''
    summary = pd.DataFrame(sv[startstopFSM.run2filter_content]).T
    r = f"{s}{summary.to_html(escape=False, index=False)}"
    _e.value = r
    #return r 
    
_a = widgets.IntText(description='No: ',layout=widgets.Layout(width='10%'))
_a.observe(html_summary, 'value')
_b = widgets.IntSlider(0, 0, rdf.shape[0]-1 , 1, layout=widgets.Layout(width='70%'))
mylink = widgets.jslink((_a, 'value'), (_b, 'value'))
_d = widgets.Button(description='Show Plots',disabled=False, button_style='primary')
_d.on_click(cb)
_e = widgets.HTML()
html_summary()

display( widgets.VBox([widgets.HBox([_b, _a, _d]),_e, out]));

In [ ]:
# PLotter
lfigures = myfigures(fsm._e)
data = pd.DataFrame([])
for doplot in ['actors','lubrication']:
#for doplot in ['actors']:
    dmaxlength = None
    dminlength = None
    #dmaxlength = 1200
    #dminlength = 200000
    dset = lfigures[doplot]
    vset = cvset(mp,dset)
    startversuch = rdb.iloc[vv]; 
    try:
        data = get_cycle_data2(fsm, startversuch, max_length=dmaxlength, min_length=dminlength, cycletime=1, silent=True, p_data=cvset(mp,dset))
        data['power_diff'] = pd.Series(np.gradient(data['Power_PowerAct']))
        fig = FSM_splot(fsm, startversuch, data, dset, figsize=(18,10))
        fig = FSM_add_Notations(fig, fsm, startversuch)
        disp_alarms(startversuch)
        disp_warnings(startversuch)
        fig = FSM_add_Alarms(fig, fsm, startversuch)
        fig = FSM_add_Warnings(fig, fsm, startversuch)
        bokeh_show(fig)
    except Exception as err:
        print(err)

In [ ]:

def xwhere(data,key,level):
    return data.iloc[data['datetime'][1:][np.array(data[key][1:]-level) * np.array(data[key][:-1]-level) < 0].index]

def xwhere2(data,key,level):
    pts = data['datetime'][1:][np.array(data[key][1:]-level) * np.array(data[key][:-1]-level) < 0]
    newlist =[x for p in list(pts.index) for x in [p-1,p]]
    return data.iloc[newlist]

def xwhere_dir(data,key,level,direction):
    data['diff'] = pd.Series(np.diff(data[key].values))
    pts = data['datetime'][1:][np.array(data[key][1:]-level) * np.array(data[key][:-1]-level) < 0]
    if direction == 'up':
        erg = data.iloc[pts.index][data['diff'] > 0]
    elif direction == 'down': 
        erg = data.iloc[pts.index][data['diff'] < 0]
    else:
        erg = data.iloc[pts.index]
    return erg

xwhere_dir(data,'Various_Values_SpeedAct',50,'up')

In [ ]:
op0 = xwhere(data,'Hyd_PressOil',1.1)
op1 = xwhere(data,'Hyd_PressOil',0.1)
op2 = xwhere(data,'Hyd_PressOil',0.5)
prelube_degasing_time = (op0.iloc[0]['time'] - op1.iloc[0]['time']) / 1000.0
post_lube_time = (op1.iloc[1]['time'] - op2.iloc[1]['time']) / (1000.0 * 60) 
prelube_degasing_time, 'sec' , post_lube_time, 'min'

In [ ]:
xwhere(data,'Hyd_PressOil',0.5)

In [ ]:
xwhere(data,'Hyd_PressOil',0.1)

In [ ]:
xwhere(data,'Various_Values_SpeedAct',1503)

In [ ]:
def cdata(startversuch, data, phases):
    ts = int(startversuch['startstoptiming'][phases[0]][0]['start'].timestamp() * 1000)
    te = int(startversuch['startstoptiming'][phases[-1]][0]['end'].timestamp() * 1000)
    return data[(data.time > ts) & (data.time < te)].reset_index(drop=True)

def extract_data(startversuch, data, dataItemName, phases):
    ts = int(startversuch['startstoptiming'][phases[0]][0]['start'].timestamp() * 1000)
    te = int(startversuch['startstoptiming'][phases[-1]][0]['end'].timestamp() * 1000)
    return data[['datetime',dataItemName]][(data.time > ts) & (data.time < te)].reset_index(drop=True)

def edges_in_phase2(startversuch, data, dataItemName, phases, direction, ax=None):
    c = extract_data(startversuch, data, dataItemName, phases)
    c['_diff'] = pd.Series(np.diff(c[dataItemName].values))
    pts = xwhere_dir(c, '_diff', c['_diff'].max() * 0.1, direction)
    if ax is not None:
        ax.plot(c['datetime'], c[dataItemName])
        for i,p in pts.iterrows():
            ax.axvline(p['datetime'],linestyle='--', c='k', alpha=0.8, linewidth=1)
        ax.grid()        
    return pts

def FSM_pre_phases(startversuch, data, dataItemName, phases, direction, fig=None, y_pos=2, color='rgba(0,0,0,0.8)', line='solid', alpha=1.0, perc=0.11):
    c = extract_data(startversuch, data, dataItemName, phases)
    c['_diff'] = pd.Series(np.diff(c[dataItemName].values))
    if direction == 'up':
        tresh_hold = c['_diff'].max()
    elif direction == 'down':
        tresh_hold = c['_diff'].min()
    pts = xwhere_dir(c, '_diff', tresh_hold * perc, direction)
    if fig is not None:
        for i,p in pts.iterrows():
            FSM_VLine(fig, txt=f"{p['datetime'].strftime('%Y-%m-%d %H:%M:%S')} {dataItemName}", x_pos=p['datetime'], y_pos=y_pos, color=color, line=line, alpha=alpha)
            #ax.axvline(p['datetime'],linestyle='--', c='k', alpha=0.8, linewidth=1)
    return fig

def FSM_start(startversuch, data, dataItemName, phases, fig=None, y_pos=2, color='rgba(0,0,0,0.8)', line='solid', alpha=1.0, perc=0.11):
    c = extract_data(startversuch, data, dataItemName, phases)
    pts = xwhere_dir(c, dataItemName, c[dataItemName].max() * perc, 'up')
    if fig is not None:
        for i,p in pts.iterrows():
            FSM_VLine(fig, txt=f"{p['datetime'].strftime('%Y-%m-%d %H:%M:%S')} {dataItemName}", x_pos=p['datetime'], y_pos=y_pos, color=color, line=line, alpha=alpha)
            #ax.axvline(p['datetime'],linestyle='--', c='k', alpha=0.8, linewidth=1)
    return fig

d0 = cdata(startversuch, data, ['startpreparation','starter','speedup','idle','synchronize','targetoperation']).iloc[:400,:]
d = cdata(startversuch, data, ['startpreparation','starter','speedup'])

#fig = plt.figure(figsize=(12,8))
#ax = fig.add_subplot()
#ax.plot('datetime','Various_Values_SpeedAct', data=d)
#play(startversuch, data, 'Hyd_PressOil',['startpreparation','starter','speedup'],'up', ax)

In [ ]:
def smooth(y,box_pts):
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth

def smooth_fft(y, cut, dt):
    w = scipy.fftpack.rfft(y)
    f = scipy.fftpack.rfftfreq(len(y),dt)
    spectrum = w**2
    cutoff_idx = spectrum < (spectrum.max()/cut)
    w2 = w.copy()
    w2[cutoff_idx] = 0
    return scipy.fftpack.irfft(w2)

In [ ]:

#d0['gHyd_PressOilDif'] = pd.Series(savitzky_golay(np.array(d0['Hyd_PressOilDif']), 51, 3)) # window size 51, polynomial order 3
#d0['gHyd_PressOilDif'] = pd.Series(savitzky_golay(np.array(d0['Hyd_PressOilDif']), 29, 3)) # window size 51, polynomial order 3
d0['gHyd_PressOilDif'] = pd.Series(savgol_filter(np.array(d0['Hyd_PressOilDif']), 29, 3)) # window size 51, polynomial order 3
d0['hHyd_PressOilDif'] = pd.Series(smooth(np.array(d0['Hyd_PressOilDif']), 13))
d0['iHyd_PressOilDif'] = pd.Series(smooth_fft(np.array(d0['Hyd_PressOilDif']), 5000, 1))
fig = plt.figure(figsize=(16,12))
ax = fig.add_subplot()
ax.plot(d0['datetime'],d0['Hyd_PressOilDif'], 'b.', alpha=0.6)
ax.plot(d0['datetime'],d0['hHyd_PressOilDif'], 'g-', lw=4)
ax.plot(d0['datetime'],d0['gHyd_PressOilDif'], 'r-', lw=2)
ax.plot(d0['datetime'],d0['iHyd_PressOilDif'], 'k-', lw=2)
ax.grid()


In [ ]:
#np.array(d0['Hyd_PressOilDif'])
data

In [ ]:
dset = lfigures['lubrication']
vset = cvset(mp,dset)
fig = FSM_splot(fsm, startversuch, d0, dset, figsize=(18,10))
fig = FSM_add_Notations(fig, fsm, startversuch)
fig = FSM_add_Alarms(fig, fsm, startversuch)
fig = FSM_add_Warnings(fig, fsm, startversuch)
#fig = FSM_pre_phases(startversuch, data, 'Hyd_PressOil',['startpreparation','starter'],'up', fig, y_pos=50, color='brown')
fig = FSM_start(startversuch, data, 'Hyd_PressOil',['startpreparation'], fig, y_pos=50, color='brown',perc=0.05)
fig = FSM_start(startversuch, data, 'Various_Values_SpeedAct',['startpreparation','starter'], fig, y_pos=50, color='blue',perc=0.05)
#fig = FSM_pre_phases(startversuch, data, 'Various_Values_SpeedAct',['startpreparation','starter','speedup'],'up', fig, y_pos=250, color='blue')
bokeh_show(fig)

In [ ]:
# Algorithm visualization
dset2 = [{'col':['Power_PowerAct','helpline'], 'ylim':(-1000,12000), 'color':['red','rgba(0,128,0,0.2)'], 'unit':'kW'},
         {'col':['Exhaust_TempCylMax','Exhaust_TempCylMin'], '_ylim':(-1000,12000), 'color':['orange','rgba(255,165,0,0.4)'], 'unit':'°C'},
         {'col':['power_diff','power_diff_help'], '_ylim':(0,5000), 'color':['rgba(255,0,0,0.2)','rgba(0,128,0,0.2)'], 'unit':'kW/s'}]
ratedload = fsm._e['Power_PowerNominal']
interestingcycles = [vv] # Bautzen M01 T902
periodfactor = 3 # times calculated period length
helplinefactor = 0.8 # overlay slope to -fac * end value 
#for i, startversuch in rdb[-2:-1].iterrows() : 
for i, startversuch in rdb.iterrows() : 
    data, pow_xmax, pow_ymax, duration, ramprate = dmyplant2.loadramp_edge_detect(fsm,startversuch, periodfactor=periodfactor, helplinefactor=helplinefactor)
    if not data.empty:
        print(f"Start: {startversuch['no']:3d} xmax: {pow_xmax}, ymax: {pow_ymax:6.0f}, duration: {duration:5.1f}, ramprate: {ramprate / ratedload * 100.0:4.2f} %/s")

        data['power_diff'] = pd.Series(np.gradient(data['Power_PowerAct']))
        #data['power_diff_help'] = pd.Series(np.gradient(data['helpline']))
        fig = FSM_splot(fsm, startversuch, data, dset2, figsize=(16,8))
        fig.add_layout(Span(location=0.0,dimension='width',x_range_name='default', y_range_name='0',line_color='black', line_dash='solid', line_alpha=0.4)) 
        fig.add_layout(Span(location=pow_ymax,dimension='width',x_range_name='default', y_range_name='0',line_color='blueviolet', line_dash='dashdot', line_alpha=0.4, line_width=2)) 
        fig.add_layout(Span(location=pow_xmax,dimension='height',line_color='blueviolet', line_dash='dashdot', line_alpha=0.4, line_width=2)) 
        fig = FSM_add_Notations(fig, fsm, startversuch)
        fig = FSM_add_Alarms(fig, fsm, startversuch)
        fig = FSM_add_Warnings(fig, fsm, startversuch)
        bokeh_show(fig)

        exh_point = data['Exhaust_TempCylMax'].idxmax()
        if exh_point == exh_point: # test for not NaN
            exh_edge = data.loc[exh_point]
            exh_ymax = data.at[exh_edge.name,'Exhaust_TempCylMax']
            exh_ymin = data.at[exh_edge.name,'Exhaust_TempCylMin']
            exh_pow  = data.at[exh_edge.name,'Power_PowerAct']
            exh_pow_ymax = data[data.datetime==pow_xmax].iloc[0]['Exhaust_TempCylMax']
        print(f"Exhaust Temp at Start: min {exh_ymin:0.1f}, max {exh_ymax:0.1f}, spread {(exh_ymax-exh_ymin):0.1f}°C at {exh_pow:0.0f}kW, {exh_pow_ymax:0.1f}°C at {pow_ymax:0.0f}kW")
        dset3 = [
            {'col':['Exhaust_TempCylMax','Exhaust_TempCylMin'], '_ylim':(-1000,12000), 'color':['orange','rgba(255,165,0,0.4)','salmon'], 'unit':'°C'},
        ]
        tdata = data[['Power_PowerAct','Exhaust_TempCylMin','Exhaust_TempCylMax']]
        tdata['Spread'] = tdata['Exhaust_TempCylMax'] - tdata['Exhaust_TempCylMin']
        fig2 = dbokeh_chart(tdata, dset3, x='Power_PowerAct', figsize=(16,8))
        fig2.add_layout(Span(location=exh_pow,dimension='height',line_color='blueviolet', line_dash='dashdot', line_alpha=0.4, line_width=2)) 
        fig2.add_layout(Span(location=exh_ymax,dimension='width',x_range_name='default', y_range_name='0',line_color='blueviolet', line_dash='dashdot', line_alpha=0.4, line_width=2)) 
        fig2.add_layout(Span(location=exh_ymin,dimension='width',x_range_name='default', y_range_name='0',line_color='blueviolet', line_dash='dashdot', line_alpha=0.4, line_width=2))
        fig2.circle(x=exh_pow, y=exh_ymax, fill_color='red', radius=30)
        fig2.circle(x=exh_pow, y=exh_ymin, fill_color='red', radius=30)
        fig2.add_layout(Span(location=pow_ymax,dimension='height',line_color='blueviolet', line_dash='dashdot', line_alpha=0.4, line_width=2)) 
        fig2.add_layout(Span(location=exh_pow_ymax,dimension='width',x_range_name='default', y_range_name='0',line_color='blueviolet', line_dash='dashdot', line_alpha=0.4, line_width=2)) 
        fig2.circle(x=pow_ymax, y=exh_pow_ymax, fill_color='red', radius=30)
        bokeh_show(fig2)
    else:
        print(f"Start: {startversuch['no']:3d} no data, no improvement possible.")

In [ ]:
data

In [ ]:
fsm.results['serviceselectortiming'];

In [ ]:
for i, v in enumerate(fsm.runlogdetail(startversuch, statechanges_only=True)):
    print(f"{i:3} {v}")

In [ ]:
for i, v in enumerate(fsm.runlogdetail(startversuch, statechanges_only=False)):
    print(f"{i:3} {v}")

In [ ]:
rdb = rda
vec = ['startpreparation','speedup','idle','synchronize','loadramp','targetload','ramprate','cumstarttime','targetoperation','rampdown','coolrun','runout']
display(_=rdb[vec].hist(bins=30,figsize=(20,20)))
#ax_list[0][2].set_xlim((0,10))
#display(rdb[vec].hist(bins=30,figsize=(20,20)))
display(rdb[vec].describe().round(2))

In [ ]:
dset2 = [{'col':['Power_PowerAct','helpline'], 'ylim':(-1000,12000), 'color':['red','rgba(0,128,0,0.2)'], 'unit':'kW'},
         {'col':['power_diff','power_diff_help'], '_ylim':(0,5000), 'color':['rgba(255,0,0,0.2)','rgba(0,128,0,0.2)'], 'unit':'kW/s'}]
ratedload = fsm._e['Power_PowerNominal']
periodfactor = 3 # times calculated period length
helplinefactor = 0.8 # overlay slope to -fac * end value 
t0 = time.time()
for i, startversuch in rdb[-1:].iterrows() : 
#for i, startversuch in tqdm(rdb.iterrows(), total=rdb.shape[0], ncols=80, mininterval=1, unit=' starts', desc="FSM Run2"):
    data, xmax, ymax, duration, ramprate = dmyplant2.loadramp_edge_detect(fsm,startversuch, periodfactor, helplinefactor)
    if not data.empty:
        print(f"Start: {startversuch['no']:3d} xmax: {xmax}, ymax: {ymax:6.0f}, duration: {duration:5.1f}, ramprate: {ramprate / ratedload * 100.0:4.2f} %/s")
        data['power_diff'] = pd.Series(np.gradient(data['Power_PowerAct']))
        #data['power_diff_help'] = pd.Series(np.gradient(data['helpline']))
        fig = FSM_splot(fsm, startversuch, data, dset2, figsize=(14,6))
        fig.add_layout(Span(location=0.0,dimension='width',x_range_name='default', y_range_name='0',line_color='black', line_dash='solid', line_alpha=0.4)) 
        fig.add_layout(Span(location=ymax,dimension='width',x_range_name='default', y_range_name='0',line_color='blueviolet', line_dash='dashdot', line_alpha=0.4, line_width=2)) 
        fig.add_layout(Span(location=xmax,dimension='height',line_color='blueviolet', line_dash='dashdot', line_alpha=0.4, line_width=2)) 
        fig = FSM_add_Notations(fig, fsm, startversuch)
        #fig = FSM_add_Alarms(fig, fsm, startversuch)
        #fig = FSM_add_Warnings(fig, fsm, startversuch)
        bokeh_show(fig)
    else:
        print(f"Start: {startversuch['no']:3d} no data, no improvement possible.")
t1 = time.time()
print(f"{(t1-t0):4.1f} sec")    

In [ ]:
mfn = e._fname + '_messages.txt'
fsm.save_messages(mfn)
print(mfn)